In [1]:
import tweepy
import pandas as pd
import os 
from dotenv import load_dotenv
import json
import csv
import requests

In [2]:
load_dotenv('apipass', verbose = True)

True

In [3]:
endpoint = os.environ.get('TWITTER_API_ENDPOINT')
consumer_key = os.environ.get('TWITTER_CONSUMER_KEY')
consumer_secret = os.environ.get('TWITTER_CONSUMER_SECRET')
bearer_token = os.environ.get('TWITTER_BEARER_TOKEN')

In [4]:
# authentication
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_token_secret)

In [5]:
# initialization of API
#api = tweepy.API(auth)
#api

In [6]:
consumer_key = "..."
access_token= "..."

client=tweepy.Client(bearer_token='...')

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

NameError: name 'token_secret' is not defined

In [16]:
auth = tweepy.Client(bearer_token)
api = tweepy.API(auth)

In [23]:
# list of variables to retrieve
variables = ['id', 'id_str', 'location', 
             'description', 'url', 'followers_count', 
             'friends_count', 'listed_count', 'created_at', 
             'favourites_count', 'verified', 'statuses_count', 
             'lang', 'status', 'default_profile', 'pinned_tweet_id',
             'default_profile_image', 'has_extended_profile', 'name', 'username']

In [24]:
# read input file with usernames
usernames = pd.read_csv('authors_metrics2.csv')['author.username'].tolist()

In [25]:
len(usernames)

12547

In [26]:
def slice_list(input_list, size):
    "Slice an input list into a specific size and yield shorten lists"
    for i in range(0, len(input_list), size):
        output_list = input_list[i:i + size]
        yield output_list
    return

In [27]:
# list to store results
results = []

In [30]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get('TWITTER_BEARER_TOKEN')


def create_url(list_100):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
    str_users = ','.join(list_100)
    usernames = f"usernames={str_users}"
    valid_fields = ['created_at', 'description', 'entities', 'id', 'location', 'name',
                    'pinned_tweet_id', 'profile_image_url', 'protected', 'public_metrics',
                    'url', 'username', 'verified', 'verified_type', 'withheld']
    user_fields = "user.fields=" + ",".join(valid_fields)
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

#### 
def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

#### remove the break transform print into create json (when it works)


In [31]:
n = 0
usernameslist = []
for list_100 in slice_list(usernames, 100):
    url = create_url(list_100)
    json_response = connect_to_endpoint(url)
    for user in json_response['data']:
        user_info = {
            'username': user['username'],
            'followers_count': user['public_metrics']['followers_count'],
            'friends_count': user['public_metrics']['following_count']
        }
        usernameslist.append(user_info)
    #usernameslist.append(json.dumps(json_response, indent=4, sort_keys=True))
    usernameslist.append(json_response)
    n += 1
    if n == 2:
        break
   

200
200


In [ ]:
usernameslist

In [ ]:
with open('output.json', 'w') as f:
    json.dump(usernameslist, f)

In [ ]:
#save json list 
#then change the variables, retry running and run without the break


In [ ]:
#output_json

In [ ]:
dfjson['public_metrics']

In [ ]:
# create new columns for each metric

dfjson['followers_count'] = dfjson['public_metrics'].apply(lambda x: x['followers_count'])
dfjson['following_count'] = dfjson['public_metrics'].apply(lambda x: x['following_count'])
dfjson['tweet_count'] = dfjson['public_metrics'].apply(lambda x: x['tweet_count'])
dfjson['listed_count'] = dfjson['public_metrics'].apply(lambda x: x['listed_count'])

# drop the original column
dfjson.drop(columns=['public_metrics'], inplace=True)


In [ ]:
dfjson.columns

In [ ]:
dfjson.to_csv('output.csv', index=False)